<a href="https://colab.research.google.com/github/Nazarique/Perceptron_Adaline/blob/main/Fundamentos_de_Redes_Neurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
class Perceptron:
  ''' Objeto Perceptron, um único neurônio para realizar operações linearmente separáveis '''
  __bias = -1
  def __init__(self, features, interactions = 100,
                learning_rate = 0.01):
    '''  Método construtor da classe Perceptron, 
          Default - interações = 100 e    
          função de ativação degrau.
    '''
    
    self.learning_rate = learning_rate
    self.weight = np.random.random_sample(features+1)
    self.interactions = interactions
    self.MSE = [1]
    
  def forward(self, x):
    u = np.dot(x, self.weight)
    return np.heaviside(u,0.)
    
  def train(self, xtrain, ytrain):
    xtrain = np.insert(xtrain, 0,   
                        Perceptron.__bias, axis=1)
    self.interaction = 0
    escape = False
    error = np.zeros(ytrain.shape[0])
    cont = 0
    
    while (escape == False): 
      escape = True
      cont = 0
      self.interaction += 1 
      
      for inputs, label in zip(xtrain, ytrain):
          yPredict = self.forward(inputs)
          error[cont] = (label - yPredict) 
          cont+=1
          
          if(yPredict!=label):
              escape = False 
              self.weight = self.weight + (
            self.learning_rate * (label - yPredict) * inputs)
              
      if self.interaction >= self.interactions: 
        escape = True
      self.MSE.append(np.square(error).mean())
    print('Quantidade de Interações: '+ str(
                                self.interaction))
    
  def test(self, x, y):  
    x = np.insert(x, 0, Perceptron.__bias, axis=1)
    yPredict = self.forward(x)
    #error = (y - yPredict) 
    #print('MSE: '+str(np.square(error).mean()))   
    return yPredict

In [ ]:
class Adaline(Perceptron):
  
  ''' Objeto Adaline, um único neurônio para realizar operações 
      linearmente separáveis, tem como herança a classe Perceptron
  '''
    
  def __init__(self, features, interactions = 100,
                learning_rate = 0.01, precision = 10e-6):
    
      super().__init__(features, interactions,
                learning_rate)
      self.precision = precision
      
  def forward(self, x):
    u = np.dot(x, self.weight)
    return u 
  
  def train(self, xtrain, ytrain):
    xtrain = np.insert(xtrain, 0, -1, axis=1)
    self.interaction = 0
    escape = False
    error = np.zeros(ytrain.shape[0])
    cont = 0
    mse = 1
    while (escape == False): 
      escape = True
      cont = 0
      self.interaction += 1 

      for inputs, label in zip(xtrain, ytrain):
        
          yPredict = self.forward(inputs)
          error[cont] = (label - yPredict) 
          cont+=1
          self.weight = self.weight + (self.learning_rate * 
                                       (label - yPredict) * inputs)
          
      self.MSE.append(np.square(error).mean())
      
      if(np.absolute(self.MSE[-1] - mse) > self.precision):
          mse = self.MSE[-1]
          escape = False 
          
      if self.interaction >= self.interactions: 
        escape = True
      
    print('Quantidade de Interações: '+ str(
                                self.interaction))

In [ ]:
import numpy as np

def tanh(x):
    ''' Função tangente hiperbólica '''
    return np.tanh(x)

def dtanh(x):
    ''' Derivada da função tangente hiperbólica '''
    return 1./(np.cosh(x)**2)


class Elman:
    ''' Elman network '''


    def __init__(self, *args):
        ''' Criando rnn usando *args, lista de argumentos com a quantidade de neurônios. 
            A rnn Elman envia seu fluxo de informação nas camadas de neurônios da seguinte forma: 
            1° entrada -> 2° escondida -> 3° contexto -> 4° escondida -> 5° saída.
        '''

        self.shape = args
        n = len(args)

        # Criando as camadas e os bias 
        self.layers = []
        self.bias = []

        # Input layer (+1 unit for bias
        #              +size of first hidden layer)
        self.layers.append(np.ones(self.shape[0]+1+self.shape[1]))
        # Hidden layer(s) + output layer
        for i in range(1,n):
            self.layers.append(np.ones(self.shape[i]))
            self.bias.append(np.ones(self.shape[i]))

        # Criandos os pesos sinápticos (aleatóriamente entre -0.25 a +0.25)
        self.weights = []
        for i in range(n-1):
            self.weights.append(np.zeros((self.layers[i].size,
                                          self.layers[i+1].size)))

        # dw Variável para armazenar o ultimo valor do peso sinaptico (for momentum)
        self.dw = [0,]*len(self.weights) # Vetor de zeros do mesmo tamanho dos pesos

        # Reset weights (Função para zerar os pesos sinápticos)
        self.reset()

    def reset(self):
        ''' Reset weights '''

        for i in range(len(self.weights)):
            Z = np.random.random((self.layers[i].size,self.layers[i+1].size))
            self.weights[i][...] = (2*Z-1)*0.25

    def forward(self, data):
        ''' Propaga os dados da camada de entrada para a camada de saída. '''

        # Inserindo os dados na camada de entrada
        self.layers[0][:self.shape[0]] = data
        # Por ser uma rnn os dados também são colocados na camada de contexto
        self.layers[0][self.shape[0]:-1] = self.layers[1]

    
        for i in range(1,len(self.shape)):

            # Somatório vetorial entre os pesos sinápticos e os dados
            soma = np.dot(self.layers[i-1],self.weights[i-1])
            # Inserindo o bias e usando uma função de ativação
            self.layers[i][...] = tanh(soma + self.bias[i-1])

        # Return output
        return self.layers[-1]


    def backward(self, target, lrate=0.01, momentum=0.1):
        ''' Backpropagation comum. '''

        deltas = []

        # Calcula o erro da saída da rede em comparação com o sinal de referência
        error = target - self.layers[-1]
        delta = error*dtanh(self.layers[-1])
        deltas.append(delta)

        # Atualizando os valores dos pesos sinápticos e do bias
        for i in range(len(self.shape)-2,0,-1):
            delta = np.dot(deltas[0],self.weights[i].T)*dtanh(self.layers[i])
            deltas.insert(0,delta)
            
        # Update weights
        for i in range(len(self.weights)):
            layer = np.atleast_2d(self.layers[i])
            delta = np.atleast_2d(deltas[i])
            dw = np.dot(layer.T,delta)
            self.weights[i] += lrate*dw + momentum*self.dw[i]
            self.bias[i] += lrate*dw.mean(axis=0)
            self.dw[i] = dw

        # Return erro quadrático médio
        return np.sqrt((error**2).sum()/len(error))

In [ ]:
#Dados
x     = np.array([[0., 0.], [1., 0.], [0., 1.], [1., 1.]])

y_AND = np.array([0., 0., 0., 1.]) #AND
y_OR  = np.array([1., 0., 0., 1.]) #OR
y_XOR = np.array([0., 1., 1., 0.]) #XOR


In [ ]:
#Perceptron 
Pmse = []
Pinteracao = []
predict = []
for i in range(10):
  print('\nTeste: '+str(i))
  p = Perceptron(2, interactions = 500)
  p.train(x, y_AND)
  Pinteracao.append(p.interaction)
  Pmse.append(p.MSE)
  #predict.append(p.test(x, 1))
  


Teste: 0
Quantidade de Interações: 13

Teste: 1
Quantidade de Interações: 23

Teste: 2
Quantidade de Interações: 1

Teste: 3
Quantidade de Interações: 2

Teste: 4
Quantidade de Interações: 11

Teste: 5
Quantidade de Interações: 4

Teste: 6
Quantidade de Interações: 3

Teste: 7
Quantidade de Interações: 1

Teste: 8
Quantidade de Interações: 8

Teste: 9
Quantidade de Interações: 6


In [ ]:
#Adaline
Amse = []                                       #salvando erro
Ainteracao = []                                 #salvando interacao de cada rede
predict = []
for i in range(10):                             #to testando 10x
  
  print('\nTeste: '+str(i))
  a = Adaline(features = 2, interactions = 500) #criando uma adaline,  com caracteristicas, e qtdd max de interações ou epocas
  a.train(x, y_AND)                             #treina o modelo
  Ainteracao.append(a.interaction)              #salvando a interação de cada teste           
  Amse.append(a.MSE)                 
  #predict.append(a.test(x_test, 1))                             #realizando teste
  


Teste: 0
Quantidade de Interações: 257

Teste: 1
Quantidade de Interações: 247

Teste: 2
Quantidade de Interações: 376

Teste: 3
Quantidade de Interações: 299

Teste: 4
Quantidade de Interações: 136

Teste: 5
Quantidade de Interações: 36

Teste: 6
Quantidade de Interações: 120

Teste: 7
Quantidade de Interações: 187

Teste: 8
Quantidade de Interações: 262

Teste: 9
Quantidade de Interações: 325


In [ ]:
#plot
import plotly.graph_objects as go
a = 7
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.linspace(0, Pinteracao[a], Pinteracao[a]), y=Pmse[a],
                    mode='lines+markers',
                    name='Perceptron'))
fig.add_trace(go.Scatter(x=np.linspace(0, Ainteracao[a], Ainteracao[a]), y=Amse[a],
                    mode='lines+markers',
                    name='Adaline'))

fig.update_layout(title='MSE x Interação',
                   xaxis_title='Interação',
                   yaxis_title='MSE')
fig.show()



In [ ]:
#planilha
import pandas as pd
df = pd.read_csv('/EstadosUnidos_diferenciaçã.csv', header=None)

media = df[3].mean()
print(media)
df.update(df[3].replace(to_replace=0, value=media))
df.to_csv(index=False)

FileNotFoundError: ignored

In [ ]:
df.head(15)

In [ ]:
#Perceptron 
Ppmse = []
Ppinteracao = []
ppredict = []
for i in range(10):
  print('\nTeste: '+str(i))
  pp = Perceptron(3, interactions = 500)
  pp.train(x_train, y_train)
  Ppinteracao.append(pp.interaction)
  Ppmse.append(pp.MSE)
  predict.append(pp.test(x_test, 1))

In [ ]:
#Adaline
aAmse = []                                       #salvando erro
aAinteracao = []                                 #salvando interacao de cada rede
apredict = []
for i in range(10):                             #to testando 10x
  
  print('\nTeste: '+str(i))
  aa = Adaline(features = 3, interactions = 500, precision = 1e-3) #criando uma adaline,  com caracteristicas, e qtdd max de interações ou epocas
  aa.train(x_train, y_train)                             #treina o modelo
  aAinteracao.append(aa.interaction)              #salvando a interação de cada teste           
  aAmse.append(aa.MSE)                 
  apredict.append(aa.test(x_test, 1))                             #realizando teste

In [ ]:
#plot
import plotly.graph_objects as go
a = 1
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.linspace(0, Ppinteracao[a], Ppinteracao[a]), y=Ppmse[a],
                    mode='lines+markers',
                    name='Perceptron'))
fig.add_trace(go.Scatter(x=np.linspace(0, aAinteracao[a], aAinteracao[a]), y=aAmse[a],
                    mode='lines+markers',
                    name='Adaline'))

fig.update_layout(title='MSE x Interação',
                   xaxis_title='Interação',
                   yaxis_title='MSE')
fig.show()


In [ ]:

network = Elman(4,20,1)
e = []
u = [0,]*len(x_train)

for i in range(200):
    n = np.random.randint(len(x_train))
    network.forward(x_train[n])
    e.append(network.backward(y_train[n], momentum=0.1))
for i in range(0,len(x_train),1):
    o = network.forward(x_train[i])
    
    #print('Teste %d: %s -> %s' % (i, x_train[i], y_train[i]))
    #print('               Network output: %s' % o +'\n')
   

In [ ]:
#plot
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.linspace(0,200,200), y=e,
                    mode='lines+markers',
                    name='Iris_Elman'))
fig.update_layout(title='Iris_Elman - 200 loops',
                   xaxis_title='Interação',
                   yaxis_title='MSE')
fig.show()

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

n = np.random.randint(len(X))

x_train, Max, Min = norm(X)
y_train, Max, Min = norm(y)

In [ ]:
#plot
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.linspace(-5,5), y=tanh(x),
                    mode='lines+markers',
                    name='tanh'))
fig.add_trace(go.Scatter(x=np.linspace(-5,5), y=dtanh(x),
                    mode='lines+markers',
                    name='dtanh'))
fig.update_layout(title='MSE x Interação',
                   xaxis_title='Interação',
                   yaxis_title='MSE')
fig.show()